In [3]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from joblib import dump, load


In [5]:
def sliding_window(imagine, dimensiuni, pas):
    """
    Generator pentru fereastra glisantă.
    imagine: Imaginea de intrare.
    dimensiuni: Dimensiunile ferestrei (lățime, înălțime).
    pas: Pasul de glisare.
    """
    inaltime, latime = imagine.shape[:2]
    w, h = dimensiuni

    for y in range(0, inaltime - h + 1, pas):
        for x in range(0, latime - w + 1, pas):
            patch = imagine[y:y + h, x:x + w]
            yield (x, y, patch)


In [6]:
from skimage.feature import hog

def extrage_caracteristici(imagine):
    """
    Extragere caracteristici folosind HOG.
    """
    imagine_gri = cv2.cvtColor(imagine, cv2.COLOR_BGR2GRAY)
    caracteristici = hog(imagine_gri, orientations=9, pixels_per_cell=(8, 8),
                         cells_per_block=(2, 2), block_norm="L2-Hys", visualize=False)
    return caracteristici


In [ ]:
path_imagini_pozitive = "teste_poz_pe_formw_redim"
path_imagini_negative = "output_negatives_redim_bun"
imagini_pozitive = []
imagini_negative = []


In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np

# Încarcă datele antrenate (ex. caracteristici și etichete)
# Încărcare și procesare imagini
X_train = []
y_train = []

# Extragere caracteristici din imagini pozitive
for imagine in imagini_pozitive:
    caracteristici = extrage_caracteristici(imagine)
    if caracteristici is not None:
        X_train.append(caracteristici)
        y_train.append(1)

# Extragere caracteristici din imagini negative
for imagine in imagini_negative:
    caracteristici = extrage_caracteristici(imagine)
    if caracteristici is not None:
        X_train.append(caracteristici)
        y_train.append(0)

# Verificare
print(f"Număr total de mostre: {len(X_train)}")
print(f"Etichete: {set(y_train)}")

if len(X_train) == 0 or len(y_train) == 0:
    raise ValueError("Setul de date este gol. Verifică procesarea imaginilor.")

# Split setul de date
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Train set: {len(X_train)}, Validation set: {len(X_valid)}")


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [44]:
import numpy as np

def nms(detectii, scoruri, threshold=0.3):
    """
    Aplica Non-Maximum Suppression.
    detectii: Listă de dreptunghiuri (x, y, w, h).
    scoruri: Listă de scoruri asociate fiecărui dreptunghi.
    threshold: Pragul de suprapunere (IoU).
    """
    if len(detectii) == 0:
        return []

    detectii = np.array(detectii)
    scoruri = np.array(scoruri)

    x1 = detectii[:, 0]
    y1 = detectii[:, 1]
    x2 = detectii[:, 0] + detectii[:, 2]
    y2 = detectii[:, 1] + detectii[:, 3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)

    idxs = np.argsort(scoruri)[::-1]
    supravietuitori = []

    while len(idxs) > 0:
        i = idxs[0]
        supravietuitori.append(i)
        idxs = idxs[1:]

        xx1 = np.maximum(x1[i], x1[idxs])
        yy1 = np.maximum(y1[i], y1[idxs])
        xx2 = np.minimum(x2[i], x2[idxs])
        yy2 = np.minimum(y2[i], y2[idxs])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs]

        idxs = idxs[overlap <= threshold]

    return [detectii[i] for i in supravietuitori]


In [45]:
def testeaza_detector(imagine, model, dimensiuni, pas):
    detectii = []
    scoruri = []
    for dim in dimensiuni:
        for (x, y, patch) in sliding_window(imagine, dim, pas):
            caracteristici = extrage_caracteristici(patch)
            scor = model.predict_proba([caracteristici])[0, 1]  # Scor pentru clasa 1 (față)
            if scor > 0.5:
                detectii.append((x, y, dim[0], dim[1]))
                scoruri.append(scor)
    detectii_finale = nms(detectii, scoruri)
    return detectii_finale


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (91047,) + inhomogeneous part.